**Copyright 2023 Google LLC.**

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2023 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<img align="left" width="150" src="https://services.google.com/fh/files/misc/ml_toast_logo.png" alt="ml_toast_logo"></img><br/><br/>

# 🍞 ML-ToAST: **M**ulti**l**ingual **To**pic Clustering of **A**ds-triggering **S**earch **T**erms

**Disclaimer: This is not an official Google product.**

**ML-ToAST** is an open-source tool that helps users cluster multilingual search terms captured from different time windows into semantically relevant topics. It helps advertisers / marketers surface the topics or *themes* their audience are interested in, so that they can tailor their marketing activities accordingly.

Under the hood, the tool relies on Google's [Universal Sentence Encoder Multilingual](https://tfhub.dev/google/universal-sentence-encoder-multilingual/3) model to generate word embeddings, applies a couple of widely-used clustering algorithms - namely [K-Means](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/compat/v1/estimator/experimental/KMeans) and [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html) (with [UMAP](https://umap-learn.readthedocs.io/en/latest/) dimensionality reduction) - to generate clusters, and then selects the most meaningful term(s) to represent each cluster.

Input is provided via a Google Sheets spreadsheet, preprocessing and clustering takes place directly in this notebook, and the resulting topics are output back into the same spreadsheet. In other words, all data remains **private** - only visible to and in the control of the tool's user.

Though the samples provided here are specific to Google Ads, the logic can quite seamlessly be tweaked to work with any other advertising platform.

## Overview

### Challenges

Advertisers spend a significant amount of time analyzing how customers are searching for and engaging with their business, in order to understand:

* Where their existing offerings resonate most; and
* How they can tailor facets of their ads, such as landing pages and creatives, to match changing consumer interest.

One of the approaches advertisers rely on is analyzing search queries against which their ads appear, categorising these queries into meaningful themes, and analyzing themes to generate insights.

> ***Topic clustering requires both time and expertise to execute properly, and may be computationally resource intensive.***

In the context of Google Ads, the [Search Terms report](https://support.google.com/google-ads/answer/2472708) for a particular account lists all the search queries that have triggered the ads defined in that account, along with performance metrics (such as clicks, impressions, etc.). It is important to note that certain search terms that don't have enough query activity are omitted from the report in order to maintain Google's standards on data privacy.

> ***Google Ads: though the [Search Terms report](https://support.google.com/google-ads/answer/2472708) can be downloaded and analyzed, it is cumbersome for advertisers to sift through thousands of queries - usually in multiple languages - to extract meaningful insights.***

Google Ads employs several different [search automation](https://services.google.com/fh/files/misc/unlock_the_power_of_search_2022.pdf) and optimization features, one of which is Broad Match (BM). Whereas other keyword matching types focus only on syntax, [Broad Match](https://support.google.com/google-ads/answer/12159290) applies matching based on semantics - the meaning conveyed by a search - in addition to syntax. BM also looks at additional signals in the account - which include landing pages, keywords in ad groups, previous searches, and more - in order to match more relevant traffic.

> ***Google Ads: the need to understand changing user interest is becoming ever more important, particularly with automated search optimization (e.g. [Broad Match](https://support.google.com/google-ads/answer/12159290)) becoming more fundamental.***

Google Ads also provides [search term insights](https://support.google.com/google-ads/answer/11386930) within its user interface (UI). Search terms that triggered ads over the past 56 days are automatically analyzed and grouped into categories and subcategories, including performance metrics.

> ***Google Ads: [search term insights](https://support.google.com/google-ads/answer/11386930) - though technologically more sophisticated than the methodology applied here - cover the past 8 weeks only, are not available beyond the UI, and cannot be collated across accounts.***

### Introducing ML-ToAST

ML-ToAST tackles the challenges mentioned above in a simple, configurable and privacy-safe way. It relies on a Google Sheets spreadsheet as input, where search terms from different lookback windows can be compared (see the ***Which search terms to extract?*** section below for more information) to uncover only those that are *new*, and/or also those that have not received enough impressions (configurable - defaults to 1000). This represents the corpus of search terms that can be further analyzed and categorized into semantically relevant topics.

Additional input groups pertaining to Google Ads Broad Match (BM) are also built and analyzed to shed some light into BM's performance.

The figure below provides an overview of the core functionality of ML-ToAST.


<center>
<img src="https://services.google.com/fh/files/misc/ml_toast_diagram.png" alt="ml_toast_diagram"></img><br>
Fig. 1. ML-ToAST Process Diagram
</center>

### Which search terms to extract?

We recommend extracting the Google Ads [Search Terms report](https://support.google.com/google-ads/answer/2472708) for the following periods:
 * **Last 30 days** (e.g. Nov 1 - Nov 30): it generally makes sense to look at the most recent search terms that triggered your ads.
 * **Previous 30/31 days** (e.g. Oct 1 - Oct 31): this helps provide information on those search terms that constitute your core business over those that are recently trending.
 * **Last 30 days last year** (e.g. Nov 1 - Nov 30 of the previous year): to account for seasonality effects (e.g. holiday season).

We also recommend restricting the extracted search terms to a subset of *related* campaigns (e.g. all campaigns for a specific *product line* or *operating domain*) rather than all campaigns in your account. This allows the models applied here to better capture how the search terms relate to one another, and therefore, extract more meaningful topics.

The report can be downloaded from the Google Ads UI in CSV format and imported into a Google Sheets spreadsheet.

*Note: if you have multiple accounts operating under the same product line or domain, you can extract search terms from those accounts as well and group them all into the same Google Sheets spreadsheet.*

## Get Started

In [ ]:
#@title Authenticate your user for this colab session
import logging
from google.colab import auth

auth.authenticate_user()
logging.getLogger().setLevel(logging.INFO)

In [ ]:
#@title Install dependencies
!pip install tensorflow-text hdbscan umap-learn

## Input and Preprocessing


In [ ]:
#@title Configurable params { run: 'auto' }

#@markdown Enter your spreadsheet ID:
spreadsheet_id = "id-goes-here" #@param {type:"string"}

#@markdown Enter the main worksheet name (which should usually contain the search terms from the last month):
input_sheet_name = "colab-input-main" #@param {type:"string"}

#@markdown Enter the prefix for any additional worksheets you would also like to analyze (e.g. search terms of the previous month, previous year, etc.):
additional_sheets_prefix = "colab-input-lookback-" #@param {type:"string"}

#@markdown Enter the name of the column that contains search terms. This value should be the same across all worksheets:
search_terms_column = "Search term" #@param {type:"string"}

#@markdown <hr>Filtering settings

#@markdown ***Check*** the checkbox to filter on new terms (i.e. compare search terms from the aforementioned lookback worksheets) and leave ***unchecked*** to analyze search terms from the main worksheet only.
filter_new_terms = True #@param {type:"boolean"}

#@markdown Enter the name of a column that contains a metric you would like to use for filtering (e.g. impressions):
filter_metric_column = "Impr." #@param {type:"string"}

#@markdown Set this to filter terms with a *metric* (e.g. impression) value lower than the input. Set to *-1* to skip filtering.
filter_metric_max_threshold = 1000 #@param {type:"integer"}

#@markdown Select the desired logical grouping (AND/OR) of the filters:
filters_grouping = "AND" #@param ["AND", "OR"]

#@markdown <hr>Advanced settings

#@markdown Enter a comma-separated list of *stop words* which should be excluded from all generated topics:
stop_words = "stop1, stop2" #@param {type:"string"}

if stop_words:
  stop_words = stop_words.replace(', ', ',').split(',')
else:
  stop_words = None

#@markdown ***Check*** the checkbox to perform hyperparameter tuning for UMAP + HDBSCAN (increases processing time by a factor of ~3). Leave ***unchecked*** to use the default clustering parameters.
hyperparameter_tuning = False #@param {type:"boolean"}

# Google Ads specific params
match_type_column = "Match type"
match_type_broad = "Broad match"
all_report_metrics = ['Clicks', 'Impr.', 'Cost']

# Validation rules
if not spreadsheet_id or not input_sheet_name or not search_terms_column:
  raise ValueError(
      'Invalid input! Please make sure at least '
      '"spreadsheet_id", "input_sheet_name" and "search_terms_column" '
      'are provided.')

In [ ]:
#@title Fetch data from the input spreadsheet
#@markdown The first row in each worksheet will be considered the **column headers** row.
import pandas as pd
import gspread
from google.auth import default

creds, _ = default()
sheets_client = gspread.authorize(creds)
spreadsheet = sheets_client.open_by_key(spreadsheet_id)

input_values = spreadsheet.worksheet(input_sheet_name).get_all_values()
additional_sheets_values = []

if filter_new_terms and additional_sheets_prefix:
  for sheet in spreadsheet.worksheets():
    if sheet.title.startswith(additional_sheets_prefix):
      additional_sheets_values.append(sheet.col_values(1))

input_pd = pd.DataFrame(input_values[1:], columns=input_values[0])

for report_metric in all_report_metrics:
  input_pd[report_metric] = pd.to_numeric(
      input_pd[report_metric].str.replace(',', ''))

print(
    f'Worksheet: {input_sheet_name}\nNumber of rows: {len(input_pd)}\n'
    'First 5 rows:')
input_pd.head()

In [ ]:
#@title Extract search terms and apply the defined filters

def add_filter(existing_filter, new_filter):
  if filters_grouping == 'AND':
    return existing_filter & new_filter
  return existing_filter | new_filter

dfs = [
    pd.DataFrame(sheet_values[1:], columns=[sheet_values[0]])
    for sheet_values in additional_sheets_values]

data_unfiltered = input_pd.copy()

series_filter = (filters_grouping == 'AND')
applied_filters = []

if dfs:
  data_unfiltered = (
      data_unfiltered.merge(pd.concat(dfs).drop_duplicates(),
                    on=search_terms_column,
                    how='left',
                    indicator=True))
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
          data_unfiltered['_merge'] == 'left_only'))
  applied_filters.append('filter_new_terms')

if filter_metric_column and filter_metric_max_threshold > 0:
  series_filter = add_filter(
      existing_filter=series_filter, new_filter=(
      data_unfiltered[filter_metric_column] <= filter_metric_max_threshold))
  applied_filters.append(
      f'filter_metric_max_threshold < {filter_metric_max_threshold}')

filtered_data = (
    data_unfiltered[series_filter] if applied_filters else data_unfiltered)

if '_merge' in filtered_data.columns:
  filtered_data = filtered_data.drop(columns='_merge')

if filter_metric_column:
  filtered_data = filtered_data.sort_values(
      by=filter_metric_column, ascending=False)

print('\n'.join([
    f'Filtered data - total number of rows: {len(filtered_data)}',
    f'Filters applied: {applied_filters}',
    (
        f"Filters logical grouping: '{filters_grouping}'"
        if len(applied_filters) > 1 else ''),
    'First 5 rows:']))
filtered_data.head()

In [ ]:
#@title Extract 'Broad Match' terms from the main worksheet
all_broad_match = input_pd.copy()
all_broad_match = all_broad_match[
    input_pd[match_type_column] == match_type_broad]

if filter_metric_column:
  all_broad_match = all_broad_match.sort_values(
      by=filter_metric_column, ascending=False)
broad_match_groups = {'all_broad_match_terms': all_broad_match}

print(
    'Extracted:\n'
    f' - All terms where "{match_type_column}" is "{match_type_broad}" '
    f'from {input_sheet_name}. Number of rows: {len(all_broad_match)}')

## Topic Clustering

In [ ]:
#@title Import the topic clustering library
!echo "Restoring working directory to root..."
%cd /content
!rm -rf ml_toast && git clone https://github.com/google/ml_toast.git
!echo "Changing working directory to ml_toast..."
%cd ml_toast

from ml_toast import topic_clustering as topic_clustering_lib

In [ ]:
#@title Use the library to determine topics for all input groups
terms_pds = {'filtered_terms': filtered_data}
for key, broad_match_group in broad_match_groups.items():
  terms_pds[key] = broad_match_group

for key, terms_pd in terms_pds.items():
  topic_clustering = topic_clustering_lib.TopicClustering(
      data_id=key,
      input_col=search_terms_column,
      stop_words=stop_words,
      do_hdbscan_hyperopt=hyperparameter_tuning)
  topics_kmeans, topics_hdbscan = topic_clustering.determine_topics(terms_pd)
  terms_pd['Topic'] = topics_kmeans
  terms_pd['Additional Topics'] = topics_hdbscan

## Output

In [ ]:
#@title Extract performance metrics for the generated topics
metrics_pds = {}

for key, terms_pd in terms_pds.items():
  for topics_type in ['Topic', 'Additional Topics']:
    cluster_metrics = pd.DataFrame()
    cluster_metrics['Topic'] = terms_pd[topics_type]
    for report_metric in all_report_metrics:
      cluster_metrics[report_metric] = terms_pd[report_metric]

    cluster_metrics = cluster_metrics.groupby(by='Topic', sort=False).agg(
        ['mean', 'median', 'min', 'max', 'std', 'var'])
    cluster_metrics.insert(loc=0, column='Topic', value=cluster_metrics.index)
    cluster_metrics.insert(
        loc=1,
        column='Count',
        value=terms_pd.groupby(by=topics_type, sort=False).count()[
            search_terms_column])
    cluster_metrics = cluster_metrics.sort_values(by='Count', ascending=False)

    metrics_pds[
        f"{key}_{topics_type.lower().replace(' ', '_')}"] = cluster_metrics

In [ ]:
#@title Write results back to the input spreadsheet
#@markdown New worksheets with the prefix **colab-** will be appended to the spreadsheet,
#@markdown or overwritten if they already exist.
from gspread_dataframe import set_with_dataframe

for key, terms_pd in terms_pds.items():
  try:
    output_sheet = spreadsheet.worksheet(f'colab-{key}-output')
    output_sheet.clear()
  except gspread.exceptions.WorksheetNotFound:
    output_sheet = spreadsheet.add_worksheet(
        f'colab-{key}-output', rows=len(terms_pd), cols=len(terms_pd.columns))

  set_with_dataframe(output_sheet, terms_pd, include_column_header=True)

for key, metrics_pd in metrics_pds.items():
  try:
    metrics_sheet = spreadsheet.worksheet(f'colab-{key}-metrics')
    metrics_sheet.clear()
  except gspread.exceptions.WorksheetNotFound:
    metrics_sheet = spreadsheet.add_worksheet(
        f'colab-{key}-metrics',
        rows=len(metrics_pd),
        cols=len(metrics_pd.columns))
  set_with_dataframe(metrics_sheet, metrics_pd, include_column_header=True)

### Visualize the generated topics in LookerStudio

ML-Toast provides a template LookerStudio dashboard to help you visualize the generated topics and quickly surface insights.

Use [this link](https://datastudio.google.com/c/u/0/reporting/df12a39a-eab3-448e-9ce2-854c05335786/page/RRIAD/preview) to create a copy of the dashboard and get started! All you would need to do is map the data sources used by the dashboard to the spreadsheet used for the input / output above.